#### Install LightRAG


In [1]:
#!pip install neo4j nano_vectordb ollama aioboto3 lightrag-hku 
#!pip install openai langchain-openai transformers torch textract
#!pip install nest_asyncio
#!pip install ragas
#!pip install nltk
#!pip install termcolor

In [2]:
import os, json
#from langchain.chains import AnalyzeDocumentChain
from langchain_openai import ChatOpenAI

from utils import OPENAI_API_KEY

os.environ['OPENAI_API_KEY'] = OPENAI_API_KEY 


In [3]:

pdf_list = ["/home/oleg/SIT/LLM/data/data_rag/s41524-023-01062-z.pdf",
    "/home/oleg/SIT/LLM//data/data_rag/s41699-023-00369-1.pdf"
           ]
         #   "https://github.com/HSE-LAMBDA/ai4material_design/tree/main/docs/CONSTRUCTOR-MOCK.md"
         #   "https://github.com/HSE-LAMBDA/ai4material_design/blob/main/docs/CONSTRUCTOR.md",
         #   "https://github.com/HSE-LAMBDA/ai4material_design/blob/main/docs/DATA.md",
         #   "https://github.com/HSE-LAMBDA/ai4material_design/blob/main/docs/ENVIRONMENT.md",
         #   "https://github.com/HSE-LAMBDA/ai4material_design/blob/main/docs/GENERATING-CONSTRUCTOR.md",
         #   "https://github.com/HSE-LAMBDA/ai4material_design/blob/main/docs/GENERATING-MOCK.md",
         #   "https://github.com/HSE-LAMBDA/ai4material_design/blob/main/docs/PILOT.md",
         #   "https://github.com/HSE-LAMBDA/ai4material_design/blob/main/docs/SPARSE-PAPER.md"
          #  "https://www.nature.com/articles/s41377-024-01407-3",
          #  "https://www.nature.com/articles/s41565-023-01407-1",
          #  "https://www.nature.com/articles/s41699-023-00369-1",
           
            

In [5]:
import os
from lightrag import LightRAG, QueryParam
from lightrag.llm import gpt_4o_mini_complete, gpt_4o_complete

#########
# Uncomment the below two lines if running in a jupyter notebook to handle the async nature of rag.insert()
# import nest_asyncio
# nest_asyncio.apply()
######### 

WORKING_DIR = "./dickens"



if not os.path.exists(WORKING_DIR):
    os.mkdir(WORKING_DIR)

rag = LightRAG(
    working_dir=WORKING_DIR,
    chunk_token_size=1000,
    llm_model_func=gpt_4o_mini_complete  # Use gpt_4o_mini_complete LLM model
    # llm_model_func=gpt_4o_complete  # Optionally, use a stronger model
)

INFO:lightrag:Logger initialized for working directory: ./dickens
INFO:lightrag:Load KV full_docs with 4 data
INFO:lightrag:Load KV text_chunks with 25 data
INFO:lightrag:Load KV llm_response_cache with 55 data
INFO:lightrag:Loaded graph from ./dickens/graph_chunk_entity_relation.graphml with 424 nodes, 173 edges
INFO:nano-vectordb:Load (411, 1536) data
INFO:nano-vectordb:Init {'embedding_dim': 1536, 'metric': 'cosine', 'storage_file': './dickens/vdb_entities.json'} 411 data
INFO:nano-vectordb:Load (173, 1536) data
INFO:nano-vectordb:Init {'embedding_dim': 1536, 'metric': 'cosine', 'storage_file': './dickens/vdb_relationships.json'} 173 data
INFO:nano-vectordb:Load (25, 1536) data
INFO:nano-vectordb:Init {'embedding_dim': 1536, 'metric': 'cosine', 'storage_file': './dickens/vdb_chunks.json'} 25 data


In [6]:

import textract
import nest_asyncio
nest_asyncio.apply()


In [7]:
for pdf in pdf_list:

    text_content = textract.process(pdf)
    rag.insert(text_content.decode('utf-8'))

INFO:lightrag:Writing graph with 424 nodes, 173 edges
INFO:lightrag:Writing graph with 424 nodes, 173 edges


In [8]:
import networkx as nx
from pyvis.network import Network

# Load the GraphML file
G = nx.read_graphml('./dickens/graph_chunk_entity_relation.graphml')

# Create a Pyvis network
net = Network(notebook=True)

# Convert NetworkX graph to Pyvis network
net.from_nx(G)

# Save and display the network
net.show('knowledge_graph.html')

knowledge_graph.html


### Load Test Q&A

In [9]:
import pandas as pd

df_ground_true = pd.read_csv("/home/oleg/SIT/LLM/data/data_rag/QA_ai4mat_2articles.csv")

questions = df_ground_true['question'].values.tolist()
answers = df_ground_true['answer'].values.tolist()


In [254]:
from io import StringIO
import re

def get_datasamples(questions, 
                    answers,
                    mode="local",
                    top_k=10,
                   ):
    contexts = []
    ideal_answers = []

    for i,q in tqdm(enumerate(questions)):
        _context = rag.query(q, param=QueryParam(mode=mode, only_need_context=True, top_k=top_k))
        # Convert the CSV string into a Pandas DataFrame
        # Regular expression to find all content between triple backticks
        pattern = r'```(.*?)```'

        # Extract all content between ```
        matches = re.findall(pattern, _context, re.DOTALL)
        context = []
        #context = CommaSeparatedListOutputParser().invoke(matches[-1])[4::2]
        ## Nodes
        try:
            df_e = pd.read_csv(StringIO(matches[0]), header=1)
            context += list(map(lambda s: s.replace('""', ' '), (df_e['entity'] + df_e['type'] + df_e['description']).to_list()))
        except :
            context += []
        ## Relationships
        try:
            df_r = pd.read_csv(StringIO(matches[1]), header=1)
            context += list(map(lambda s: s.replace('""', ' '), (df_r['source'] + df_r['description'] + df_r['target']).to_list()))
        except :
            context += []
        ## Text
        try:
            df_c = pd.read_csv(StringIO(matches[2]), header=1)
            context += df_c['content'].to_list()
        except :
            context += []
        #context = "\n".join(context)

        i_answer = rag.query(q, param=QueryParam(mode=mode, top_k=top_k))
    
        #contexts.append(context)
        contexts.append(context)
        ideal_answers.append(i_answer)

    data_samples = {'question': questions, 
           'ground_truth': answers,
           "contexts": contexts,
          "answer": ideal_answers
          }
    return Dataset.from_dict(data_samples)


In [233]:
from ragas.metrics import (
    #context_utilization,
    #context_entity_recall,
    #context_relevancy,
    answer_relevancy,
    answer_correctness, 
    faithfulness,
    context_precision,
    context_recall,
)

from datasets import Dataset 

from ragas.evaluation import evaluate


In [234]:
metrics = [
   answer_relevancy,
    answer_correctness, 
   faithfulness,
    context_precision,
    context_recall,
 #   context_entity_recall,
 #   context_relevancy,
 #   context_utilization,
]

In [240]:
dataset, result = dict(), dict()

In [241]:
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

In [255]:
from tqdm import tqdm
for mode in ['local', 'global', 'hybrid', 'naive']:
    dataset[mode] = get_datasamples(questions,answers, mode=mode)
    
    print(mode)
    result[mode] = evaluate(dataset[mode], metrics, llm=llm,  raise_exceptions=False, embeddings=OpenAIEmbeddings())

0it [00:00, ?it/s]INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:lightrag:Local query uses 10 entites, 12 relations, 3 text units
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:lightrag:Local query uses 10 entites, 12 relations, 3 text units
1it [00:01,  1.78s/it]INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:lightrag:Local query uses 10 entites, 13 relations, 4 text units
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:lightrag:Local query uses 10 entites, 13 relations, 4 text units
2it [00:03,  1.71s/it]INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:lightrag:Local query uses 10 entites, 14 relations, 3 text units
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:lightrag:Local query uses 10 entites, 14 relations, 3 text units
3it [00:

local


Evaluating:   0%|          | 0/175 [00:00<?, ?it/s]

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"

global


...

Evaluating:   0%|          | 0/175 [00:00<?, ?it/s]

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"

naive


Evaluating:   0%|          | 0/175 [00:00<?, ?it/s]

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"

### Results

In [256]:
import ragas
def print_dict(d, indent=0):
    """
    Recursively prints a dictionary, including nested dictionaries, 
    with indentation to represent structure.
    """
    if isinstance(d, ragas.dataset_schema.EvaluationResult):
        _d = d._repr_dict.copy()
    else:
        _d = d.copy()
    for key, value in _d.items():
        print('  ' * indent + str(key) + ": ", end="")
        if isinstance(value, dict):
            print()  # Print a newline for nested dictionaries
            print_dict(value, indent + 1)
        else:
            print(str(value))

In [257]:
df = pd.DataFrame()

for k in result.keys():
    print(k)
    print_dict(result[k])
    df[k] = result[k]._repr_dict
    print()

df

local
answer_relevancy: 0.5986192317022173
answer_correctness: 0.4113072836588398
faithfulness: 0.688760854786609
context_precision: 0.08816786629063252
context_recall: 0.618095238095238

global
answer_relevancy: 0.618861552959112
answer_correctness: 0.41481908100505516
faithfulness: 0.6879827396355588
context_precision: 0.08902544563693406
context_recall: 0.6351190476190476

hybrid
answer_relevancy: 0.6565285238238676
answer_correctness: 0.488513105567387
faithfulness: 0.1125187905074053
context_precision: 0.0
context_recall: 0.11428571428571428

naive
answer_relevancy: 0.6295814045264221
answer_correctness: 0.41957449852216616
faithfulness: 0.12835978835978834
context_precision: 0.0
context_recall: 0.08571428571428572



,local,global,hybrid,naive
answer_relevancy,0.598619,0.618862,0.656529,0.629581
answer_correctness,0.411307,0.414819,0.488513,0.419574
faithfulness,0.688761,0.687983,0.112519,0.128360
context_precision,0.088168,0.089025,0.000000,0.000000
context_recall,0.618095,0.635119,0.114286,0.085714


In [258]:
from termcolor import colored

# Print colored text with termcolor

for i,r in dataset['local'].to_pandas().iterrows():
    #print(i, "\n" ,10*"*** ")
    print(colored(r['question'], "red", attrs=["bold"]))
    print(r['answer'])
    print()

* Which materials are in the dataset?
## Overview of Materials in the Dataset

The dataset focuses predominantly on two-dimensional (2D) materials, particularly highlighting several notable compounds that are widely studied within material science. Key materials featured in the dataset include:

1. **MoS₂ (Molybdenum Disulfide)**: A widely researched 2D material known for its semiconducting properties and applications in fields such as electronics, photonics, and energy storage.

2. **WSe₂ (Tungsten Diselenide)**: Similar to MoS₂, WSe₂ is another significant semiconductor with a strong light-matter interaction, making it relevant for optoelectronic devices.

3. **hBN (Hexagonal Boron Nitride)**: Often referred to as "white graphene," hBN offers excellent thermal and chemical stability, along with a wide bandgap, making it an ideal insulating layer in various applications.

4. **GaSe (Gallium Selenide)**: A lesser-known but important semiconductor, GaSe has significant optical and elect

In [259]:
from termcolor import colored

# Print colored text with termcolor

for i,r in dataset['global'].to_pandas().iterrows():
    #print(i, "\n" ,10*"*** ")
    print(colored(r['question'], "red", attrs=["bold"]))
    print(r['answer'])
    print()

* Which materials are in the dataset?
The dataset primarily focuses on a variety of two-dimensional materials, notably:

### 1. **MoS2 (Molybdenum Disulfide)**
MoS2 is recognized for its unique electrical and optical properties, making it a significant material in research surrounding defect properties and applications in electronics. It is a popular binary transition metal dichalcogenide which features prominently in defect studies.

### 2. **WSe2 (Tungsten Diselenide)**
This material exhibits semiconducting properties and is often compared with MoS2. WSe2 is frequently analyzed for its electronic behavior and defect structures, contributing to its utilization in optoelectronic devices.

### 3. **GaSe (Gallium Selenide)**
GaSe is studied for its electronic and optical characteristics. It has been included in the dataset, particularly for investigations into its defect-related properties.

### 4. **hBN (Hexagonal Boron Nitride)**
Valued for its insulating properties, hBN is included in

In [260]:
from termcolor import colored

# Print colored text with termcolor

for i,r in dataset['hybrid'].to_pandas().iterrows():
    #print(i, "\n" ,10*"*** ")
    print(colored(r['question'], "red", attrs=["bold"]))
    print(r['answer'])
    print()

* Which materials are in the dataset?
The dataset discussed in the context of two-dimensional (2D) materials includes several notable materials, each characterized for their defect properties. The primary materials included are:

### 1. **MoS2 (Molybdenum Disulfide)**
MoS2 is a transition metal dichalcogenide known for its unique electrical and optical properties. It is extensively studied in materials science, particularly in relation to its defect configurations and performance metrics under various conditions.

### 2. **WSe2 (Tungsten Diselenide)**
WSe2 is another transition metal dichalcogenide that displays semiconducting properties. It is recognized for its potential use in electronic and optoelectronic devices, making it a significant subject of study, especially regarding its defect structures.

### 3. **hBN (Hexagonal Boron Nitride)**
Hexagonal boron nitride, often abbreviated as hBN, is a 2D material valued for its insulating properties and unique electronic characteristics. 

In [261]:
from termcolor import colored

# Print colored text with termcolor

for i,r in dataset['naive'].to_pandas().iterrows():
    #print(i, "\n" ,10*"*** ")
    print(colored(r['question'], "red", attrs=["bold"]))
    print(r['answer'])
    print()

* Which materials are in the dataset?
The dataset, referred to as the machine learning friendly 2D material defect database (2DMD), includes structures with point defects for several widely used 2D materials. The materials in the dataset are:

1. MoS2 (Molybdenum Disulfide)
   - Defect types: Substitutions (S→Se, Mo→W), Vacancies (Mo, S)

2. WSe2 (Tungsten Diselenide)
   - Defect types: Substitutions (Se→S, W→Mo), Vacancies (W, Se)

3. Hexagonal Boron Nitride (h-BN)
   - Defect types: Substitutions (B→C, N→C), Vacancies (B, N)

4. GaSe (Gallium Selenide)
   - Defect types: Substitutions (Ga→In, Se→S), Vacancies (Ga, Se)

5. InSe (Indium Selenide)
   - Defect types: Substitutions (In→Ga, Se→S), Vacancies (In, Se)

6. Black Phosphorus (BP)
   - Defect types: Substitutions (P→N), Vacancies (P)

The dataset captures a variety of point defect types across these materials, facilitating the exploration of defect properties essential for developing predictive models using machine learning.

* 